### Working with unstructured data (Module 07)


In [1]:
from ptb import TreebankWordTokenizer
# This is the Penn Tree Bank tokenizer from NLTK as just one file
tok = TreebankWordTokenizer()

#### Adding a covariate

- In text processing, a variable that occurs along with words (but is not a word) is sometimes called a covariate 

- This is similar to the idea of "metadata"

- Example covariates: day of the week, time of day, number of upvotes, author, political party 

Questions: 
    
- Make a term document matrix for `libertarian.jsonl`
- Make a term document matrix for `socialism.jsonl`
- This is mostly review from last time.
- You should share the vocabularies between these two subreddits
- There are two changes

### Raw counts 

- Instead of a binary matrix, replace the 1s with the counts of each word
- So if a word occurs 5x in a document, the number should be 5 in the term-document matrix

### Stop words 

Before we start, we will add one new thing, stop words. A stop word is a common word that is excluded from analysis in text processing. In NLP, it is common to exclude stop words. There are many stop word lists out there. We will use a [common list](https://gist.github.com/sebleier/554280) from NLTK. 

Start off by downloading the list using requests. Hint: click raw on Github to get a link to the raw data.

Add in all of the punctuation in the string module

In [2]:
import requests
import string

url = "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords"
r = requests.get(url).text
stop_words = r.split("\n") + [i for i in string.punctuation]
stop_words = [i.lower() for i in stop_words]

- When you read in the tokens from your file, this time ignore the stop words. Lower case the word to see if it is a stop word 

In [3]:
import json

def get_D_and_V(fn):
    V = set()
    D = 0

    with open(fn, "r") as inf:
        for doc in inf:
            D += 1
            doc = json.loads(doc)
            for token in tok.tokenize(doc["body"]):
                if token.lower() not in stop_words:
                    V.add(token)

    V = list(V) # we want a consistent order. Not sure the latest on Python set ordering
    
    return D, V

D1, V1 = get_D_and_V(fn="libertarian.jsonl")
D2, V2 = get_D_and_V(fn="socialism.jsonl")

V = list(set(V1 + V2))

n2v = {k:v for k, v in enumerate(V)}
v2n = {v:k for k, v in enumerate(V)}

### Make a tdm for each subreddit, sharing vocabulary

In [4]:
import numpy as np
import pandas as pd

def make_tdm(_D, _V, fn):

    out = np.zeros((D2, len(_V)))

    with open(fn, "r") as inf:
        for docno, doc in enumerate(inf):
            doc = json.loads(doc)
            for token in tok.tokenize(doc["body"]):
                if token.lower() not in stop_words:
                    out[docno][v2n[token]] += 1

    out = pd.DataFrame(data=out, columns=V)
    
    return out

tdm_libertarian_df =  make_tdm(D1, V, fn="libertarian.jsonl")
tdm_socialism_df =  make_tdm(D2, V, fn="socialism.jsonl")

In [5]:
### Add in covariates, and make a big dataframe

tdm_libertarian_df["source_reddit"] = "libertarian"
tdm_socialism_df["source_reddit"] = "socialism"

tdm = pd.concat([tdm_libertarian_df, tdm_socialism_df])

#### Question 

- What does the tdm represent? 

### Question 

- What are the top terms, based on raw count in the libertarian subreddit?
- What are the top terms, based on raw count in the socialism subreddit?
- Do you think it helps that you removed stop words?
- Can you think of ways to expand the stop word list that might help?

In [6]:
counts_socialism = tdm[tdm["source_reddit"] == "socialism"]
counts_socialism = counts_socialism.drop(columns=["source_reddit"], axis=1)
counts_socialism = counts_socialism.sum(axis=0)
counts_socialism = pd.DataFrame(counts_socialism)
counts_socialism = counts_socialism.rename(columns={0: "frequency"})
counts_socialism.sort_values(axis=1, by="frequency", ascending=False, inplace=False)

,frequency
incursion,0.0
owe,1.0
Ignore,2.0
misuse.,1.0
foment,0.0
...,...
Substantial,0.0
poured,3.0
impounding.,0.0
"11,000",0.0


In [45]:
counts_libertarian = tdm[tdm["source_reddit"] == "libertarian"]
counts_libertarian = counts_libertarian.drop(columns=["source_reddit"], axis=1)
counts_libertarian = counts_libertarian.sum(axis=0)
counts_libertarian = pd.DataFrame(counts_libertarian)
counts_libertarian = counts_libertarian.rename(columns={0: "frequency"})
counts_libertarian.sort_values(axis=1, by="frequency", ascending=False , inplace=False)

,frequency
incursion,1.0
owe,9.0
Ignore,2.0
misuse.,0.0
foment,1.0
...,...
Substantial,1.0
poured,5.0
impounding.,1.0
"11,000",3.0


In [52]:
mx = np.sum(tdm.drop(["source_reddit"], axis=1)> 0)
D = tdm.shape[0]
idf = pd.DataFrame({"idf": 1/mx})

idf[idf["idf"] > .25] = 0

In [55]:
counts_libertarian["tfidf"] = counts_libertarian["frequency"] * idf["idf"]
counts_socialism["tfidf"] = counts_socialism["frequency"] * idf["idf"]

In [56]:
counts_socialism.sort_values("tfidf", ascending=False)[0:100].index

Index(['Mr', 'feat.', 'Turkish', 'hip', 'Trots', 'ft.', 'hop.', 'Poland',
       'Bookchin', 'EU', 'Harvey', 'cites', 'Community', 'absentee', 'Kliman',
       'Bolivar', 'Rosa', 'deed', 'Proudhon', 'Berlin', 'bourgeosie', 'coca',
       'Proyect', 'hop', 'Bioshock', 'FARC', 'LTRPF', 'er', 'stereotypes',
       'Hamas', 'Eurozone', 'moneyless', 'intrinsically', 'Costs', 'Rise',
       'Verso', 'Tijoux', 'soc', 'Buddhist', 'marxists', 'delusions',
       'left-liberal', 'Scottish', 'WSWS', 'Technique', 'Immortal', 'Cuban',
       'Labour', 'Yugoslavia', 'Guevara', 'rap', 'parliaments', 'IRC', 'Venus',
       'editors', 'battalion', 'uncompromising', 'Alice', 'tribalism', 'USSR.',
       'obesity', 'vouchers', 'Cuba', 'organising', 'NATO', 'blah', 'Marxism',
       'innovation.', 'Vox', 'PKK', 'halls', 'accumulate', 'Comunista',
       'Reconstrucción', 'patriarchal', 'comfort/entertainment', 'achievable',
       'Ana', 'Feminists', 'deficit', 'Mos', 'rewarding', 'Lupe', 'Buddhism',
    

In [57]:
counts_libertarian.sort_values("tfidf", ascending=False)[0:100].index

Index(['^|', 'Nexium', 'quot', 'hijackers', 'mg', '009', 'ESA', 'glasses',
       'Prilosec', 'Gilded', 'NRA', 'passengers', 'temp', 'amp', 'DNA',
       'airlines', '039', 'incestuous', '/message/compose', 'to=autowikibot',
       'Conley', 'WTC', '^or', 'OTC', 'incest', 'message=', '^delete',
       'subject=AutoWikibot', 'herd', 'stickers', 'FISA', 'prescription',
       '*****', 'soda', 'ID', 'sorority', 'Barr', '^libertarianism', 'FICA',
       '2F', 'MJ', 'elective', '^of', 'encryption', 'plane.', 'Gardner',
       'exchanges', 'tl', '//en.wikipedia.org/wiki/Libertarianism', 'refund',
       'charter', 'sexuality.', 'heroin', 'Musk', 'temps', 'Pentagon',
       'marriage', 'lenses', 'airline', 'GM', 'planes', 'Kasich',
       'announcement', 'Air', 'Towers', 'nihilist', 'nuisance', 'vaccination',
       'Flight', 'statute', 'Biden', 'blah', '10th', 'standard.', 'resolved',
       '14th', 'evaluating', 'dwelling', 'SWAT', 'joints', 'Josh', 'weed.',
       'encrypted', 'Richmond', 

In [51]:
counts_libertarian.sort_values("tfidf", ascending=True)[0:100]

,frequency,tfidf
incursion,1.0,0.0
//gizmodo.com/the-fbis-secret-aircraft-are-watching-us-1702799186,1.0,0.0
instructive,0.0,0.0
means-tested,1.0,0.0
gulag-worthy,0.0,0.0
...,...,...
Ebert,0.0,0.0
boys,8.0,0.0
//i.imgur.com/UcjMaRB.png,0.0,0.0
falsified,1.0,0.0


In [17]:
idf.loc['people']

idf    1.884862
Name: people, dtype: float64

In [32]:
idf["idf"].sort_values(ascending=False)[0:10]

2000's.                              1.0
epistopsychological                  1.0
//www.democracyatwork.info/radio/    1.0
breakbeats                           1.0
interventions.                       1.0
Imho                                 1.0
limousine                            1.0
Continued                            1.0
dipping                              1.0
accelerates                          1.0
Name: idf, dtype: float64